<a href="https://colab.research.google.com/github/GiulioAppetito/Progetto_MOBD_2122/blob/main/prova3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/My\ Drive/ProgettoMOBD

/content/drive/My Drive/ProgettoMOBD


In [9]:
import sklearn
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn_pandas import CategoricalImputer
from sklearn.ensemble import IsolationForest, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
from sklearn.svm import SVC, LinearSVC
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn import tree
import graphviz
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

np.random.seed = 123

In [10]:
# LETTURA DATASET
data = pd.read_csv("train.csv", sep=",")
print(data.head())
# print("\nLabels: {}".format(np.unique(data.Y, return_counts=True)))

# print("\n# of NaNs values for each column:")
# print(data.isnull().sum(axis=0))

x = data.iloc[:, :-1].to_numpy()
y = data.iloc[:, -1].to_numpy()
# print(x.shape)
# print(y.shape)

# stratify: to mantain the same mean as y for y_tr and y_ts
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.3, random_state=258, stratify=y)

# print(x_tr.shape)
# print(y_tr.shape)
# print(x_ts.shape)
# print(y_ts.shape)

# print(np.unique(y, return_counts=True)[1]/len(y))
# print(np.unique(y_tr, return_counts=True)[1]/len(y_tr))
# print(np.unique(y_ts, return_counts=True)[1]/len(y_ts))

   F0  F1      F2  F3  F4  F5  F6  F7         F8      F9   F10  F11  F12  \
0  39  K1   77516  R1  13  Q1  P1  M1  caucasian    Male  2174    0   40   
1  50  K2   83311  R1  13  Q2  P2  M2  caucasian    Male     0    0   13   
2  38  K3  215646  R2   9  Q3  P3  M1  caucasian    Male     0    0   40   
3  53  K3  234721  R3   7  Q2  P3  M2      black    Male     0    0   40   
4  28  K3  338409  R1  13  Q2  P4  M3      black  Female     0    0   40   

    F13  Y  
0   USA  0  
1   USA  0  
2   USA  0  
3   USA  0  
4  Cuba  0  


In [26]:
# valori nan
imp = SimpleImputer(missing_values=np.nan, strategy = 'most_frequent')
x_tr_notnan = imp.fit_transform(x_tr)
x_ts_notnan = imp.transform(x_ts)

#gestione features categoriche
#dato che però questi sono ndarrays, per fare la one hot coded technique dobbiamo usare dataframes

x_tr_df = pd.DataFrame(x_tr_notnan)
x_ts_df = pd.DataFrame(x_ts_notnan)

x_tr_df.columns=['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13']
x_ts_df.columns=['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13']

# one hot code

final_df = pd.concat([x_tr_df,pd.get_dummies(x_tr_df['F1'])],axis = 1)
final_df.drop('F1', axis=1, inplace=True)

final_df = pd.concat([final_df,pd.get_dummies(final_df['F7'])],axis = 1)
final_df.drop('F7', axis=1, inplace=True)

final_df = pd.concat([final_df,pd.get_dummies(final_df['F9'])],axis = 1)
final_df.drop('F9', axis=1, inplace=True)

final_df = pd.concat([final_df,pd.get_dummies(final_df['F8'])],axis = 1)
final_df.drop('F8', axis=1, inplace=True)

final_df = pd.concat([final_df,pd.get_dummies(final_df['F5'])],axis = 1)
final_df.drop('F5', axis=1, inplace=True)

final_df = pd.concat([final_df,pd.get_dummies(final_df['F6'])],axis = 1)
final_df.drop('F6', axis=1, inplace=True)

final_df = pd.concat([final_df,pd.get_dummies(final_df['F13'])],axis = 1)
final_df.drop('F13', axis=1, inplace=True)

final_df.drop('F3', axis=1, inplace=True)

print(final_df.head())


   F0      F2  F4 F10 F11 F12  K1  K2  K3  K4  ...  Poland  Portugal  \
0  27  130067  10   0   0  65   0   0   1   0  ...       0         0   
1  18  703067   7   0   0  20   0   0   1   0  ...       0         0   
2  33  144949  13   0   0  65   0   0   0   0  ...       0         0   
3  54  176681   9   0   0  20   0   0   1   0  ...       0         0   
4  39  115076  10   0   0  40   0   0   1   0  ...       0         0   

   Puerto-Rico  Scotland  South  Taiwan  Thailand  USA  Vietnam  Yugoslavia  
0            0         0      0       0         0    1        0           0  
1            0         0      0       0         0    1        0           0  
2            0         0      0       0         0    1        0           0  
3            0         0      0       0         0    1        0           0  
4            0         0      0       0         0    1        0           0  

[5 rows x 88 columns]
